In [1]:
import argparse
import psycopg2
import csv
from configparser import ConfigParser

def connect_to_db():
    try:
        connection = psycopg2.connect(
            user="gpuser",
            password="gpuser@123",
            host="192.168.4.127",
            port="5432",
            database="UniversalNetworkDataHub_Jamaica"
        )
        return connection
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
        return None


In [ ]:
import psycopg2
import csv
import argparse
import ast

# Hardcoded data list
data = [
    {
        "Table": "L_UeMeasControl",
        "Column": "ueMeasurementsActiveCDMA2000",
        "Column_Golden_Value": 0
    },
    {
        "Table": "L_UeMeasControl",
        "Column": "bothA5RsrpRsrqCheck",
        "Column_Golden_Value": 1
    },
    {
        "Table": "L_UeMeasControl",
        "Column": "inhibitB2RsrqConfig",
        "Column_Golden_Value": 0
    },
    {
        "Table": "L_QciProfilePredefined",
        "Column": "tReorderingUl",
        "Column_Golden_Value": "35 ",
        "Join_Column": "DimensionId",
        "Condition_Table_1": "L_QciProfilePredefined",
        "Condition_Table_1_Column": "qciProfilePredefinedId",
        "Condition_Table_1_Column_Value": "qci1"
    },
    {
        "Table": "L_QciProfilePredefined",
        "Column": "ulMinBitRate",
        "Column_Golden_Value": "0 ",
        "Join_Column": "DimensionId",
        "Condition_Table_1": "L_QciProfilePredefined",
        "Condition_Table_1_Column": "qciProfilePredefinedId",
        "Condition_Table_1_Column_Value": "qci1"
    },
    {
        "Table": "L_QciProfilePredefined",
        "Column": "relativePriority",
        "Column_Golden_Value": 3,
        "Join_Column": "DimensionId",
        "Condition_Table_1": "L_QciTable",
        "Condition_Table_1_Column": "qciTableId",
        "Condition_Table_1_Column_Value": "default",
        "Condition_Table_2": "L_QciProfilePredefined",
        "Condition_Table_2_Column": "qciProfilePredefinedId",
        "Condition_Table_2_Column_Value": "qci7"
    },
    {
        "Table": "L_QciProfilePredefined",
        "Column": "relativePriority",
        "Column_Golden_Value": 3,
        "Join_Column": "DimensionId",
        "Condition_Table_1": "L_QciTable",
        "Condition_Table_1_Column": "qciTableId",
        "Condition_Table_1_Column_Value": "default",
        "Condition_Table_2": "L_QciProfilePredefined",
        "Condition_Table_2_Column": "qciProfilePredefinedId",
        "Condition_Table_2_Column_Value": "qci8"
    },
    {
        "Table": "L_QciProfilePredefined",
        "Column": "dlResourceAllocationStrategy",
        "Column_Golden_Value": "FREQUENCY_SELECTIVE",
        "Join_Column": "DimensionId",
        "Condition_Table_1": "L_QciTable",
        "Condition_Table_1_Column": "qciTableId",
        "Condition_Table_1_Column_Value": "default",
        "Condition_Table_2": "L_QciProfilePredefined",
        "Condition_Table_2_Column": "qciProfilePredefinedId",
        "Condition_Table_2_Column_Value": "qci8",
        "Condition_Table_3": "Dimensions",
        "Condition_Table_3_Column": "Market",
        "Condition_Table_3_Column_Value": "Panama"
    },
    {
        "Table": "L_QciProfilePredefined",
        "Column": "schedulingAlgorithm",
        "Column_Golden_Value": "PROPORTIONAL_FAIR_LOW",
        "Join_Column": "DimensionId",
        "Condition_Table_1": "L_QciTable",
        "Condition_Table_1_Column": "qciTableId",
        "Condition_Table_1_Column_Value": "default",
        "Condition_Table_2": "L_QciProfilePredefined",
        "Condition_Table_2_Column": "qciProfilePredefinedId",
        "Condition_Table_2_Column_Value": "qci8",
        "Condition_Table_3": "Dimensions",
        "Condition_Table_3_Column": "Market",
        "Condition_Table_3_Column_Value": "Panama"
    }
]

# Function to establish database connection
def connect_to_db():
    try:
        connection = psycopg2.connect(user="your_username",
                                      password="your_password",
                                      host="your_host",
                                      port="your_port",
                                      database="your_database")
        cursor = connection.cursor()
        return connection, cursor
    except psycopg2.OperationalError as e:
        print(f"Error: {e}")
        return None, None

# Function to execute SQL query and fetch results
def execute_query(cursor, query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except psycopg2.Error as e:
        print(f"Error: {e}")
        return []

# Function to write results to CSV file
def write_to_csv(results):
    try:
        with open('audit_results.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Level', 'Table', 'Column', 'Golden_Value', 'Current_Value'])
            writer.writerows(results)
    except IOError as e:
        print(f"Error writing to CSV: {e}")

# Function to construct SQL query based on conditions
def construct_query(table, entry, timestamp, sites):
    try:
        conditions = []
        if 'Join_Column' in entry:
            for i in range(1, 4):
                condition_table = entry.get(f"Condition_Table_{i}")
                condition_column = entry.get(f"Condition_Table_{i}_Column")
                condition_value = entry.get(f"Condition_Table_{i}_Column_Value")
                if condition_table and condition_column and condition_value:
                    conditions.append((condition_table, condition_column, condition_value))

        query = f"SELECT \"{table}\".\"{entry['Column']}\" FROM \"{table}\""

        if conditions:
            for i, (condition_table, condition_column, condition_value) in enumerate(conditions):
                query += f"""
                JOIN "{condition_table}" AS ct{i} ON "{table}"."{entry['Join_Column']}" = ct{i}."DimensionId" AND ct{i}."{condition_column}" = '{condition_value}'
                """

        if timestamp:
            if "WHERE" in query:
                query += f" AND \"{table}\".\"timestamp\" = '{timestamp}'"
            else:
                query += f" WHERE \"{table}\".\"timestamp\" = '{timestamp}'"

        if sites:
            site_conditions = " OR ".join([f'"{table}"."site" = \'{site}\'' for site in sites])
            if "WHERE" in query:
                query += f" AND ({site_conditions})"
            else:
                query += f" WHERE {site_conditions}"

        return query
    except KeyError as e:
        print(f"KeyError: {e}")
        return None

# Main function to handle
parser = argparse.ArgumentParser(description='Database Golden Value Audit Script')
parser.add_argument('--tables', type=str, default='["all"]', help='List of tables in the form ["table1", "table2", ...]')
parser.add_argument('--columns', type=str, default='[]', help='List of columns in the form ["column1", "column2", ...]')
parser.add_argument('--timestamp', type=str, default=None, help='Timestamp value')
parser.add_argument('--site', type=str, default='[]', help='List of sites in the form ["site1", "site2", ...]')
args = parser.parse_args()

tables = ast.literal_eval(args.tables)
columns = ast.literal_eval(args.columns)
site = ast.literal_eval(args.site)
timestamp = args.timestamp